In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

In [22]:
df = pd.read_pickle('result/debug/pred_ep_9.pkl.gz')

In [23]:
df.head()

,target,pred,subset
0,"[[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0...","[[[0.77100587, 0.90346026, 0.94780254, 0.98387...",training
1,"[[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0...","[[[0.80034035, 0.9072118, 0.9647836, 0.9830999...",training
2,"[[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0...","[[[0.87005967, 0.959478, 0.9879616, 0.98997366...",training
3,"[[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0...","[[[0.7816798, 0.9170533, 0.96449065, 0.9741969...",training
4,"[[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0...","[[[0.77857625, 0.91274285, 0.9703845, 0.988032...",training


In [24]:
row = df.iloc[0]

In [25]:
target = row['target']
pred = row['pred']

In [26]:
print(target.shape)
print(pred.shape)

(5, 50, 50)
(5, 50, 50)


In [35]:
px.imshow(target[4, :, :])

In [36]:
px.imshow(pred[4, :, :])